In [21]:
import geopandas as gpd
import pandas as pd
from scipy.spatial import cKDTree
import numpy as np
import xlrd
import fiona as fi
import rtree

In [22]:
#read in necessary data

boulder_income = gpd.read_file(r'C:\Users\arom\Documents\School\Grad School\Fall 2020\Automation\Project\data\BoulderCensusIncome\BoulderCensusIncome\Boulder county Census Tract Income.shp')
trails = gpd.read_file(r'C:\Users\arom\Documents\School\Grad School\Fall 2020\Automation\Project\data\Boulder_Area_Trails-shp\Boulder_Area_Trails.shp')
bus_data = pd.read_excel(r'C:\Users\arom\Documents\School\Grad School\Fall 2020\Automation\Project\data\bus_book2.xlsx') 
th_data = pd.read_excel(r'C:\Users\arom\Documents\School\Grad School\Fall 2020\Automation\Project\data\th_book.xlsx')

In [23]:
#reproject

boulder_income = boulder_income.to_crs("EPSG:4326")
trails = trails.to_crs("EPSG:4326")

In [24]:
#create arrays of coordinate pairs for the bus stations and trailheads

bus_df = pd.DataFrame(bus_data, columns= ['LAT', 'LNG'])
bus_array = bus_df.to_numpy()

th_df = pd.DataFrame(th_data, columns= ['LAT', 'LNG'])
th_array = th_df.to_numpy()

In [25]:
#confirm array shape

bus_array.shape

(1264, 2)

In [26]:
#confirm array shape

th_array.shape

(326, 2)

In [27]:
#run nearest neighbor to get distance between trailhead and nearest bus stop

dist, i = cKDTree(bus_array).query(th_array)

In [28]:
#confirm distance array shape, one distance per trailhead

dist.shape

(326,)

In [29]:
#print closest and farthest distance from bus stop to nearest trailhead, NOTE: 364567.2 is the conversion from degrees to feet

print('The nearest trailhead to a bus stop is:',np.min(dist)*364567.2,'feet, and the farthest trailhead from a bus stop is:',np.max(dist)*364567.2,'feet')

The nearest trailhead to a bus stop is: 22.905889836192877 feet, and the farthest trailhead from a bus stop is: 194461.1244870641 feet


In [30]:
#compute how many trailheads are in each tract

#tracts with boulder income data
boulder_income = gpd.GeoDataFrame.from_file(r'C:\Users\arom\Documents\School\Grad School\Fall 2020\Automation\Project\data\BoulderCensusIncome\BoulderCensusIncome\Boulder county Census Tract Income.shp')
ths = gpd.GeoDataFrame.from_file(r'C:\Users\arom\Documents\School\Grad School\Fall 2020\Automation\Project\data\Boulder_Area_Trailheads-shp\Boulder_Area_Trailheads.shp')

# empty list to get count of points
pts_in_polys = []

#Loop through polygons
for i, poly in boulder_income.iterrows():

    #keep a list of points in each poly
    pts_in_this_poly = []

    #loop over all points
    for j, pt in ths.iterrows():
        if poly.geometry.contains(pt.geometry):
            #add to list
            pts_in_this_poly.append(pt.geometry)
            ths = ths.drop([j])

    #append the number of points to list
    pts_in_polys.append(len(pts_in_this_poly))

#Add the number of points for each poly to the dataframe.
boulder_income['number of points'] = gpd.GeoSeries(pts_in_polys)

C:\Users\arom\anaconda3\envs\geog5092_37\lib\site-packages\ipykernel_launcher.py:27: FutureWarning:     You are passing non-geometry data to the GeoSeries constructor. Currently,
    it falls back to returning a pandas Series. But in the future, we will start
    to raise a TypeError instead.


In [31]:
#check how many trailheads are in each tract

pts_in_polys

[0,
 1,
 0,
 0,
 3,
 0,
 4,
 2,
 0,
 0,
 14,
 0,
 0,
 2,
 1,
 0,
 13,
 2,
 30,
 0,
 4,
 3,
 5,
 4,
 0,
 9,
 2,
 1,
 35,
 3,
 14,
 21,
 2,
 0,
 13,
 0,
 7,
 0,
 0,
 3,
 5,
 1,
 0,
 2,
 4,
 1,
 0,
 0,
 1,
 1,
 0,
 12,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 7,
 0,
 17,
 0,
 1,
 2]

In [32]:
#convert from geodataframe to dataframe

boulder_income_df = pd.DataFrame(boulder_income)

In [33]:
#look at max values of trailheads and income, note: Poverty__1 is mean income for census tract

maxValues = boulder_income_df.max() 
maxValues

OBJECTID                                                   439
FIPS                                               08013061400
Tract_Name          Census Tract 614, Boulder County, Colorado
Tract_FIPS                                         08013061400
County                                                 BOULDER
Population                                               12041
Populati_1                                             19082.1
Percent_Po                                                63.4
Poverty_Me                                              152433
Poverty__1                                              174068
Poverty_Pe                                               67676
number of points                                            35
dtype: object

In [34]:
#allow pandas to show entire dataframe

pd.set_option('display.max_rows', None)

In [35]:
#create table to show wealth of tract with number of trailheads in tract

final_df = pd.DataFrame(boulder_income, columns= ['Poverty__1', 'number of points'])
final_df

,Poverty__1,number of points
0,125654,0
1,51640,1
2,57492,0
3,106277,0
4,78326,3
5,84311,0
6,68243,4
7,92830,2
8,59195,0
9,64747,0


In [36]:
#join trails with boulder income to summarize total trail distance, in miles, in each tract

trail_join = gpd.sjoin(trails, boulder_income, how='inner', op='within')

trail_join.groupby('Tract_Name').agg('sum')

,OBJECTID_left,LENGTH,SHAPESTLen,index_right,OBJECTID_right,Population,Populati_1,Percent_Po,Poverty_Me,Poverty__1,Poverty_Pe,number of points
Tract_Name,,,,,,,,,,,,
"Census Tract 121.01, Boulder County, Colorado",33352,0.927838,1950.765951,352,6208,74960,68145.6,150.4,1592784,2409632,1015696,80
"Census Tract 121.02, Boulder County, Colorado",197294,2.452351,5158.868444,4140,29808,507771,457449.3,641.7,5069913,8416275,3508167,0
"Census Tract 121.03, Boulder County, Colorado",316764,3.212497,6760.657018,6600,43800,395000,443820.0,840.0,8945500,13865300,5868800,100
"Census Tract 121.04, Boulder County, Colorado",543151,8.029425,16889.960195,13144,92008,526608,506362.0,2056.4,22894092,28454852,12023792,1484
"Census Tract 121.05, Boulder County, Colorado",231647,1.910866,4021.264401,4636,32908,426664,444440.4,623.2,6141788,7700472,3310408,0
"Census Tract 122.01, Boulder County, Colorado",193097,6.850106,14389.818711,2150,33626,336690,164242.8,2614.4,5697500,9661842,4292346,774
"Census Tract 122.02, Boulder County, Colorado",256885,1.832876,3853.539267,4838,35342,493640,783559.2,3517.8,3118624,4789702,2543722,0
"Census Tract 122.03, Boulder County, Colorado",1315433,16.597943,34894.266863,1708,157990,3017609,1207043.6,5721.8,23013592,33445202,18893469,1281
"Census Tract 122.04, Boulder County, Colorado",314586,2.361658,4963.049139,3696,44688,456512,845387.2,5073.6,4464768,8939056,4236176,0
